## Functions

In [1]:

from LyndonWords import *
import itertools
from tqdm.notebook import tqdm
import math
import numpy as np
import cProfile
def genPermutations(type,n,k):
    rootSystemPermutations = np.empty(math.factorial(n+1),dtype=object)
    v = 0
    for i in tqdm(itertools.permutations(list(range(n+1))),total=math.factorial(n+1)):
        rootSystemPermutations[v] = rootSystem(i,type,k)
        v+=1
    return rootSystemPermutations

In [2]:
def checkMonotoneIncreasingImpDeltaStart(rootSystemPermutations):
    for rootsys in rootSystemPermutations:
        returnArr = []
        deltaWords = rootsys.getWords(rootsys.delta)
        temp = rootsys.checkMonotonicity("Increasing")
        monotoneIncreasing = [rootsys.getWords(i[0] + rootsys.delta)[0] for i in temp]
        deltaArrays = [i.string for i in deltaWords]
        for i in monotoneIncreasing:
            flag = False
            for j in deltaWords:
                if(str(i).startswith(str(j))):
                    flag = True
                    break
            if(flag):
                continue
            for j in rootsys.getAffineWords(i.weights):
                if np.any(np.all(j[0:sum(rootsys.delta)] == deltaArrays)):
                    returnArr.append((rootsys.ordering,j))
    return returnArr

In [3]:
def checkDeltaStartImpMonotoneIncreasing(rootSystemPermutations):
    returnArr = []
    for rootsys in rootSystemPermutations:
        deltaWords = rootsys.getWords(rootsys.delta)
        deltaArrays = [i.string for i in deltaWords]
        for base in rootsys.baseWeights[:-1]:
            secondWord = rootsys.getWords(base + rootsys.delta)[0]
            flag = False
            for i in deltaArrays:
                if(np.all(secondWord[:rootsys.deltaDegree] == i)):
                    flag = True
                    break
            if not flag:
                continue
            if(rootsys.getMonotonicity(base) != 1):
                returnArr.append((str(rootsys.ordering),str(rootsys.getWords(base)[0])))
    return returnArr

In [4]:
def printByLength(rootSystem):
    len = 0
    for k in range(rootSystem.k + 1):
        for i in rootSystem.baseWeights:
            comb = i + rootSystem.delta * k
            if(sum(comb) != len):
                len = sum(comb)
                print(len)
            for i in rootSystem.getWords(comb):
                print(" "+str(i))

In [5]:
def printByBaseWord(rootSystem:rootSystem,comb):
    k=0
    isImaginary = rootSystem.isImaginary(comb)
    for i in rootSystem.getAffineWords(comb):
        if(isImaginary and k % rootSystem.n == 0):
            print(k //rootSystem.n)
        if(not isImaginary):
            print(k)
        print(" " + str(i))
        k+=1

In [6]:
def exportByLength(rootSystem,quoteMarks=False):
    file = open("out.txt",'w')
    file.write(str(rootSystem.ordering)+"\n")
    for k in range(rootSystem.k + 1):
        for i in rootSystem.baseWeights:
            comb = i + rootSystem.delta * k
            for i in rootSystem.getWords(comb):
                if(quoteMarks):
                    file.write(f"'{str(i)}',")
                else:
                    file.write(str(i) + '\n')
    file.close()

In [7]:
def checkDeltaPattern(perms,k=2,printDelta = False):
    for i in perms:
        deltaWords = i.getWords(k*i.delta)
        flag = True
        for j in range(1,len(deltaWords)):
            if(deltaWords[j][sum(i.delta)-1] != deltaWords[j][0]):
                if(flag):
                    print(i.ordering)
                    flag = False
                if(printDelta):
                    print(f"  SL_{j}(\delta): " + '-'.join([str(i) for i in i.costandardFactorization(i.getWords(i.delta)[j])]))
                print(f"  SL_{j}({k}*\delta): " + '-'.join([str(i) for i in i.costandardFactorization((deltaWords[j]))]))

In [8]:
def checkHDeltasRepeat(perms):
    excepts = []
    for i in perms:
        hs = [j.hs for j in i.getAffineWords(i.delta)]
        bases = hs[i.n:2*i.n]
        for offset in range(0,i.n):
            for k in range(2,len(hs)//i.n):
                if(np.any(bases[offset] != hs[offset + k*i.n])):
                    excepts.append((str(i.ordering),f"SL_{offset+1}({k}\delta)"))
    return excepts

In [9]:
exportByLength(rootSystem([1,4,5,2,3,0],'D',3),True)

## Type C

In [10]:
Cperms = genPermutations('C',5,3)

  0%|          | 0/720 [00:00<?, ?it/s]

In [11]:
exceptions = checkMonotoneIncreasingImpDeltaStart(Cperms)
if(len(exceptions) != 0):
    print("Exception")

Above shows that monotone increasing implies it starts with a delta word

In [12]:
checkDeltaStartImpMonotoneIncreasing(Cperms)

[('0<1<2<3<4<5', '1'),
 ('0<1<2<3<4<5', '2'),
 ('0<1<2<3<4<5', '3'),
 ('0<1<2<3<4<5', '4'),
 ('0<1<2<3<4<5', '5'),
 ('0<1<2<3<4<5', '4,5'),
 ('0<1<2<3<4<5', '1,2'),
 ('0<1<2<3<4<5', '2,3'),
 ('0<1<2<3<4<5', '3,4'),
 ('0<1<2<3<4<5', '3,4,5'),
 ('0<1<2<3<4<5', '1,2,3'),
 ('0<1<2<3<4<5', '2,3,4'),
 ('0<1<2<3<4<5', '4,4,5'),
 ('0<1<2<3<4<5', '2,3,4,5'),
 ('0<1<2<3<4<5', '3,4,5,4'),
 ('0<1<2<3<4<5', '1,2,3,4'),
 ('0<1<2<3<4<5', '1,2,3,4,5'),
 ('0<1<2<3<4<5', '2,3,4,5,4'),
 ('0<1<2<3<4<5', '3,4,3,4,5'),
 ('0<1<2<3<4<5', '1,2,3,4,5,4'),
 ('0<1<2<3<4<5', '2,3,4,5,4,3'),
 ('0<1<2<3<4<5', '1,2,3,4,5,4,3'),
 ('0<1<2<3<4<5', '2,3,4,2,3,4,5'),
 ('0<1<2<3<4<5', '1,2,3,4,5,4,3,2'),
 ('0<1<2<3<4<5', '1,2,3,4,1,2,3,4,5'),
 ('0<1<2<3<5<4', '1'),
 ('0<1<2<3<5<4', '2'),
 ('0<1<2<3<5<4', '3'),
 ('0<1<2<3<5<4', '4'),
 ('0<1<2<3<5<4', '5'),
 ('0<1<2<3<5<4', '5,4'),
 ('0<1<2<3<5<4', '1,2'),
 ('0<1<2<3<5<4', '2,3'),
 ('0<1<2<3<5<4', '3,4'),
 ('0<1<2<3<5<4', '3,4,5'),
 ('0<1<2<3<5<4', '1,2,3'),
 ('0<1<2<3<5<4',

In [13]:
C205134 = rootSystem([2,0,5,1,3,4],'C',5)
for i in C205134.getAffineWords([0,0,1,0,0,0]):
    print(*C205134.costandardFactorization(i),sep='-')


3-None
2,3,1,0,1-2,3,4,5,4,3
2,3,1,0,1,2,3,4,1,0,2,3,4,1,5-2,3,4,5,4,3
2,3,1,0,1,2,3,4,1,0,2,3,4,1,5,2,3,4,1,0,2,3,4,1,5-2,3,4,5,4,3
2,3,1,0,1,2,3,4,1,0,2,3,4,1,5,2,3,4,1,0,2,3,4,1,5,2,3,4,1,0,2,3,4,1,5-2,3,4,5,4,3
2,3,1,0,1,2,3,4,1,0,2,3,4,1,5,2,3,4,1,0,2,3,4,1,5,2,3,4,1,0,2,3,4,1,5,2,3,4,1,0,2,3,4,1,5-2,3,4,5,4,3


Implies that beginning with a k-delta word implies monotone increasing

In [14]:
checkDeltaPattern(Cperms,3,printDelta=True)

2<0<3<1<4<5
  SL_4(\delta): 2,3,4,5,4,3-2,1,0,1
  SL_4(3*\delta): 2,3,4,5,4,3-2,1,0,1,2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5
2<0<3<1<5<4
  SL_4(\delta): 2,3,4,5,4,3-2,1,0,1
  SL_4(3*\delta): 2,3,4,5,4,3-2,1,0,1,2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5
2<0<3<4<1<5
  SL_4(\delta): 2,3,4,5,4,3-2,1,0,1
  SL_4(3*\delta): 2,3,4,5,4,3-2,1,0,1,2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5
2<0<3<4<5<1
  SL_4(\delta): 2,3,4,5,4,3-2,1,0,1
  SL_4(3*\delta): 2,3,4,5,4,3-2,1,0,1,2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5
2<0<3<5<1<4
  SL_4(\delta): 2,3,4,5,4,3-2,1,0,1
  SL_4(3*\delta): 2,3,4,5,4,3-2,1,0,1,2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5
2<0<3<5<4<1
  SL_4(\delta): 2,3,4,5,4,3-2,1,0,1
  SL_4(3*\delta): 2,3,4,5,4,3-2,1,0,1,2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5
2<0<4<1<3<5
  SL_3(\delta): 2,3,4,5,4-2,3,1,0,1
  SL_3(3*\delta): 2,3,4,5,4-2,3,1,0,1,2,3,1,4,0,2,3,1,4,5,2,3,1,4,0,2,3,1,4,5
  SL_4(\delta): 2,1,0,1-2,3,4,5,4,3
  SL_4(3*\delta): 2,1,0,1-2,3,4,5,4,3,2,3,1,4,0,2,3,1,4,5,2,3,1,4,0,2,3,1,4,5
2<0<

In [15]:
checkHDeltasRepeat(Cperms)

[]

In [16]:
index=0
[print(*list(Cperms[index].costandardFactorization(i))+[i.hs],sep='-',) for i in Cperms[index].getAffineWords(Cperms[index].delta)]

0,1,2,3,4,5,4,3,2-1-[1 0 0 0 0]
0,1,2,3,4,5,4,3-1,2-[1 1 0 0 0]
0,1,2,3,4,5,4-1,2,3-[1 1 1 0 0]
0,1,2,3,4,5-1,2,3,4-[1 1 1 1 0]
0-1,2,3,4,1,2,3,4,5-[2 2 2 2 1]
0,1,2,3,4,5,4,3,2-0,1,2,3,4,5,4,3,2,1,1-[1 0 0 0 0]
0,1,2,3,4,5,4,3,1-0,1,2,3,4,5,4,3,2,1,2-[0 1 0 0 0]
0,1,2,3,4,5,4,1,2-0,1,2,3,4,5,4,3,1,2,3-[0 0 1 0 0]
0,1,2,3,4,5,1,2,3-0,1,2,3,4,5,4,1,2,3,4-[0 0 0 1 0]
0,1,2,3,4,1,2,3,4-0,1,2,3,4,5,1,2,3,4,5-[0 0 0 0 1]
0,1,2,3,4,5,4,3,2-0,1,2,3,4,5,4,3,2,1,0,1,2,3,4,5,4,3,2,1,1-[1 0 0 0 0]
0,1,2,3,4,5,4,3,1-0,1,2,3,4,5,4,3,2,1,0,1,2,3,4,5,4,3,2,1,2-[0 1 0 0 0]
0,1,2,3,4,5,4,1,2-0,1,2,3,4,5,4,3,1,2,0,1,2,3,4,5,4,3,1,2,3-[0 0 1 0 0]
0,1,2,3,4,5,1,2,3-0,1,2,3,4,5,4,1,2,3,0,1,2,3,4,5,4,1,2,3,4-[0 0 0 1 0]
0,1,2,3,4,1,2,3,4-0,1,2,3,4,5,1,2,3,4,0,1,2,3,4,5,1,2,3,4,5-[0 0 0 0 1]
0,1,2,3,4,5,4,3,2-0,1,2,3,4,5,4,3,2,1,0,1,2,3,4,5,4,3,2,1,0,1,2,3,4,5,4,3,2,1,1-[1 0 0 0 0]
0,1,2,3,4,5,4,3,1-0,1,2,3,4,5,4,3,2,1,0,1,2,3,4,5,4,3,2,1,0,1,2,3,4,5,4,3,2,1,2-[0 1 0 0 0]
0,1,2,3,4,5,4,1,2-0,1,2,3,4,5,4,3,1,

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

## Type G

In [11]:
G2perms = genPermutations('G',2,5)

  0%|          | 0/6 [00:00<?, ?it/s]

In [12]:
checkDeltaStartImpMonotoneIncreasing(G2perms)

[('0<1<2', '1'),
 ('0<1<2', '2'),
 ('0<1<2', '1,2'),
 ('0<1<2', '1,2,2'),
 ('0<1<2', '1,2,2,2'),
 ('0<1<2', '1,2,1,2,2'),
 ('0<2<1', '1'),
 ('0<2<1', '2'),
 ('0<2<1', '2,1'),
 ('0<2<1', '2,2,1'),
 ('0<2<1', '2,2,2,1'),
 ('0<2<1', '2,2,1,2,1'),
 ('1<0<2', '0'),
 ('1<0<2', '1,2,2'),
 ('1<0<2', '1,2,2,0'),
 ('1<2<0', '2'),
 ('1<2<0', '1,0'),
 ('1<2<0', '1,0,2'),
 ('1<2<0', '1,0,2,2'),
 ('1<2<0', '1,0,2,2,2'),
 ('2<0<1', '0'),
 ('2<0<1', '2,1'),
 ('2<0<1', '2,1,0'),
 ('2<0<1', '2,2,1,0'),
 ('2<0<1', '2,1,2,1,0'),
 ('2<1<0', '0'),
 ('2<1<0', '2,1'),
 ('2<1<0', '2,1,0'),
 ('2<1<0', '2,2,1,0'),
 ('2<1<0', '2,1,2,1,0')]

In [13]:
checkMonotoneIncreasingImpDeltaStart(G2perms)

[]

In [14]:
for i in G2perms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [15]:
index = 4
print(str(G2perms[index].ordering))

printByBaseWord(G2perms[index],G2perms[index].delta)

2<0<1


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
checkDeltaPattern(G2perms)

In [ ]:
checkHDeltasRepeat(G2perms)

[]

## Type B

In [16]:
Bperms = genPermutations('B',5,4)

  0%|          | 0/720 [00:00<?, ?it/s]

In [17]:
checkDeltaStartImpMonotoneIncreasing(Bperms)

[('0<1<2<3<4<5', '1'),
 ('0<1<2<3<4<5', '2'),
 ('0<1<2<3<4<5', '3'),
 ('0<1<2<3<4<5', '4'),
 ('0<1<2<3<4<5', '5'),
 ('0<1<2<3<4<5', '4,5'),
 ('0<1<2<3<4<5', '1,2'),
 ('0<1<2<3<4<5', '2,3'),
 ('0<1<2<3<4<5', '3,4'),
 ('0<1<2<3<4<5', '3,4,5'),
 ('0<1<2<3<4<5', '1,2,3'),
 ('0<1<2<3<4<5', '2,3,4'),
 ('0<1<2<3<4<5', '4,5,5'),
 ('0<1<2<3<4<5', '2,3,4,5'),
 ('0<1<2<3<4<5', '1,2,3,4'),
 ('0<1<2<3<4<5', '3,4,5,5'),
 ('0<1<2<3<4<5', '1,2,3,4,5'),
 ('0<1<2<3<4<5', '2,3,4,5,5'),
 ('0<1<2<3<4<5', '3,4,5,5,4'),
 ('0<1<2<3<4<5', '1,2,3,4,5,5'),
 ('0<1<2<3<4<5', '2,3,4,5,5,4'),
 ('0<1<2<3<4<5', '1,2,3,4,5,5,4'),
 ('0<1<2<3<4<5', '2,3,4,5,5,4,3'),
 ('0<1<2<3<4<5', '1,2,3,4,5,5,4,3'),
 ('0<1<2<3<4<5', '1,2,3,4,5,5,4,3,2'),
 ('0<1<2<3<5<4', '1'),
 ('0<1<2<3<5<4', '2'),
 ('0<1<2<3<5<4', '3'),
 ('0<1<2<3<5<4', '4'),
 ('0<1<2<3<5<4', '5'),
 ('0<1<2<3<5<4', '5,4'),
 ('0<1<2<3<5<4', '1,2'),
 ('0<1<2<3<5<4', '2,3'),
 ('0<1<2<3<5<4', '3,4'),
 ('0<1<2<3<5<4', '3,4,5'),
 ('0<1<2<3<5<4', '1,2,3'),
 ('0<1<2<3<5<4',

In [18]:
checkMonotoneIncreasingImpDeltaStart(Bperms)

[]

In [19]:
for i in Bperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [20]:
checkDeltaPattern(Bperms,printDelta=True)

2<0<5<1<3<4
  SL_3(\delta): 2,3,1,0-2,3,4,5,5,4
  SL_3(2*\delta): 2,3,1,0-2,3,4,5,5,0,2,3,4,1,2,3,4,5,5,4
2<0<5<1<4<3
  SL_3(\delta): 2,3,1,0-2,3,4,5,5,4
  SL_3(2*\delta): 2,3,1,0-2,3,4,5,5,0,2,3,4,1,2,3,4,5,5,4
2<1<5<0<3<4
  SL_3(\delta): 2,3,0,1-2,3,4,5,5,4
  SL_3(2*\delta): 2,3,0,1-2,3,4,5,5,1,2,3,4,0,2,3,4,5,5,4
2<1<5<0<4<3
  SL_3(\delta): 2,3,0,1-2,3,4,5,5,4
  SL_3(2*\delta): 2,3,0,1-2,3,4,5,5,1,2,3,4,0,2,3,4,5,5,4
2<4<0<1<3<5
  SL_4(\delta): 2,1,0-2,3,4,5,5,4,3
  SL_4(2*\delta): 2,1,0-2,3,4,5,5,4,3,2,3,0,4,5,5,2,3,1,4
2<4<0<1<5<3
  SL_4(\delta): 2,1,0-2,3,4,5,5,4,3
  SL_4(2*\delta): 2,1,0-2,3,4,5,5,4,3,2,3,0,4,5,5,2,3,1,4
2<4<0<5<1<3
  SL_4(\delta): 2,1,0-2,3,4,5,5,4,3
  SL_4(2*\delta): 2,1,0-2,3,4,5,5,4,3,2,3,0,4,5,5,2,3,1,4
2<4<1<0<3<5
  SL_4(\delta): 2,0,1-2,3,4,5,5,4,3
  SL_4(2*\delta): 2,0,1-2,3,4,5,5,4,3,2,3,1,4,5,5,2,3,0,4
2<4<1<0<5<3
  SL_4(\delta): 2,0,1-2,3,4,5,5,4,3
  SL_4(2*\delta): 2,0,1-2,3,4,5,5,4,3,2,3,1,4,5,5,2,3,0,4
2<4<1<5<0<3
  SL_4(\delta): 2,0,1-2,3,4,5,5,4,

In [21]:
checkHDeltasRepeat(Bperms)

[]

## Type D

In [22]:
Dperms = genPermutations('D',5,4)

  0%|          | 0/720 [00:00<?, ?it/s]

In [23]:
checkMonotoneIncreasingImpDeltaStart(Dperms)

[]

In [24]:
checkDeltaStartImpMonotoneIncreasing(Dperms)

[('0<1<2<3<4<5', '1'),
 ('0<1<2<3<4<5', '2'),
 ('0<1<2<3<4<5', '3'),
 ('0<1<2<3<4<5', '4'),
 ('0<1<2<3<4<5', '5'),
 ('0<1<2<3<4<5', '1,2'),
 ('0<1<2<3<4<5', '2,3'),
 ('0<1<2<3<4<5', '3,4'),
 ('0<1<2<3<4<5', '3,5'),
 ('0<1<2<3<4<5', '1,2,3'),
 ('0<1<2<3<4<5', '2,3,4'),
 ('0<1<2<3<4<5', '2,3,5'),
 ('0<1<2<3<4<5', '3,5,4'),
 ('0<1<2<3<4<5', '1,2,3,4'),
 ('0<1<2<3<4<5', '1,2,3,5'),
 ('0<1<2<3<4<5', '2,3,5,4'),
 ('0<1<2<3<4<5', '1,2,3,5,4'),
 ('0<1<2<3<4<5', '2,3,5,4,3'),
 ('0<1<2<3<4<5', '1,2,3,5,4,3'),
 ('0<1<2<3<4<5', '1,2,3,5,4,3,2'),
 ('0<1<2<3<5<4', '1'),
 ('0<1<2<3<5<4', '2'),
 ('0<1<2<3<5<4', '3'),
 ('0<1<2<3<5<4', '4'),
 ('0<1<2<3<5<4', '5'),
 ('0<1<2<3<5<4', '1,2'),
 ('0<1<2<3<5<4', '2,3'),
 ('0<1<2<3<5<4', '3,4'),
 ('0<1<2<3<5<4', '3,5'),
 ('0<1<2<3<5<4', '1,2,3'),
 ('0<1<2<3<5<4', '2,3,4'),
 ('0<1<2<3<5<4', '2,3,5'),
 ('0<1<2<3<5<4', '3,4,5'),
 ('0<1<2<3<5<4', '1,2,3,4'),
 ('0<1<2<3<5<4', '1,2,3,5'),
 ('0<1<2<3<5<4', '2,3,4,5'),
 ('0<1<2<3<5<4', '1,2,3,4,5'),
 ('0<1<2<3<5<4', '2

In [25]:
checkDeltaPattern(Dperms,2,printDelta=True)

In [26]:
for i in Dperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [28]:
checkHDeltasRepeat(Dperms)

[]

## Type F

In [ ]:
Fperms = genPermutations('F',4,4)

  0%|          | 0/720 [00:00<?, ?it/s]

In [ ]:
checkMonotoneIncreasingImpDeltaStart(Fperms)

[]

In [ ]:
checkDeltaStartImpMonotoneIncreasing(Fperms)

[('0<1<2<3<4<5', '1'),
 ('0<1<2<3<4<5', '2'),
 ('0<1<2<3<4<5', '3'),
 ('0<1<2<3<4<5', '4'),
 ('0<1<2<3<4<5', '5'),
 ('0<1<2<3<4<5', '1,2'),
 ('0<1<2<3<4<5', '2,3'),
 ('0<1<2<3<4<5', '3,4'),
 ('0<1<2<3<4<5', '3,5'),
 ('0<1<2<3<4<5', '1,2,3'),
 ('0<1<2<3<4<5', '2,3,4'),
 ('0<1<2<3<4<5', '2,3,5'),
 ('0<1<2<3<4<5', '3,5,4'),
 ('0<1<2<3<4<5', '1,2,3,4'),
 ('0<1<2<3<4<5', '1,2,3,5'),
 ('0<1<2<3<4<5', '2,3,5,4'),
 ('0<1<2<3<4<5', '1,2,3,5,4'),
 ('0<1<2<3<4<5', '2,3,5,4,3'),
 ('0<1<2<3<4<5', '1,2,3,5,4,3'),
 ('0<1<2<3<4<5', '1,2,3,5,4,3,2'),
 ('0<1<2<3<5<4', '1'),
 ('0<1<2<3<5<4', '2'),
 ('0<1<2<3<5<4', '3'),
 ('0<1<2<3<5<4', '4'),
 ('0<1<2<3<5<4', '5'),
 ('0<1<2<3<5<4', '1,2'),
 ('0<1<2<3<5<4', '2,3'),
 ('0<1<2<3<5<4', '3,4'),
 ('0<1<2<3<5<4', '3,5'),
 ('0<1<2<3<5<4', '1,2,3'),
 ('0<1<2<3<5<4', '2,3,4'),
 ('0<1<2<3<5<4', '2,3,5'),
 ('0<1<2<3<5<4', '3,4,5'),
 ('0<1<2<3<5<4', '1,2,3,4'),
 ('0<1<2<3<5<4', '1,2,3,5'),
 ('0<1<2<3<5<4', '2,3,4,5'),
 ('0<1<2<3<5<4', '1,2,3,4,5'),
 ('0<1<2<3<5<4', '2

In [ ]:
checkDeltaPattern(Fperms,2,printDelta=True)

In [ ]:
for i in Fperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [ ]:
checkHDeltasRepeat(Fperms)

[]